# CIFAR Conv Net

И так, в этом ноутбуке Вы сделаете превую в своей жизни сверточную сеть! На сложном датасете. Cкачайте его кстати, 

In [1]:
# !mkdir cifar10
# !curl -o cifar-10-python.tar.gz https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
# !tar -xvzf cifar-10-python.tar.gz -C cifar10

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage.interpolation import rotate
from lasagne.layers import batch_norm
import cv2

%matplotlib inline

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 980 (CNMeM is disabled, cuDNN 5105)


In [3]:
from cifar import load_CIFAR10
plt.rcParams['figure.figsize'] = (10.0, 8.0) 

cifar10_dir = './cifar10/cifar-10-batches-py'
X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)
X_train = X_train.astype(np.float32)
y_train = y_train.astype(np.float32)
X_test = X_test.astype(np.float32)
y_test = y_test.astype(np.float32)

In [4]:
# classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
# num_classes = len(classes)
# samples_per_class = 7
# for y, cls in enumerate(classes):
#     idxs = np.flatnonzero(y_train == y)
#     idxs = np.random.choice(idxs, samples_per_class, replace=False)
#     for i, idx in enumerate(idxs):
#         plt_idx = i * num_classes + y + 1
#         plt.subplot(samples_per_class, num_classes, plt_idx)
#         plt.imshow(X_train[idx].astype('uint8').transpose(1, 2, 0))
#         plt.axis('off')
#         if i == 0:
#             plt.title(cls)
# plt.show()

# augmentation

In [5]:
augmentation = []
augmentation_y = []
len1 = len(X_train)
angles = [0, 90, 180, 270]
for img, lbl in zip(X_train, y_train):
    img0 = img.astype('uint8').transpose(1, 2, 0)
    augmentation.append(img0.astype('uint8').transpose(2, 0, 1))
    augmentation.append(cv2.flip(img0, 1).astype('uint8').transpose(2, 0, 1))
    augmentation_y.append(lbl)
    augmentation_y.append(lbl)
    
#     rows,cols = (32, 32)
#     for a in [90, -90]:
#         M = cv2.getRotationMatrix2D((cols/2,rows/2),a,1)
#         dst = cv2.warpAffine(img0,M,(cols,rows))
#         dst = dst.astype('uint8').astype('uint8').transpose(2, 0, 1)
#         augmentation.append(dst)
#         augmentation_y.append(lbl)
X_train = np.array(augmentation)
y_train = np.array(augmentation_y)

<h1 align="center">First of all -- Checking Questions</h1> 

**Вопрос 1**: Чем отличаются современные сверточные сети от сетей 5 летней давности?

<Ответ>

**Вопрос 2**: Какие неприятности могут возникнуть во время обучения современных нейросетей?

<Ответ>

**Вопрос 3**: У вас есть очень маленький датасет из 100 картинок, но вы очень хотите использовать нейросеть, какие неприятности вас ждут и как их решить?

<Ответ>

**Вопрос 4**: У вас есть очень маленький датасет из 100 картинок, классификация, но вы очень хотите использовать нейросеть, какие неприятности вас ждут и как их решить? что делать если первый вариант  решения не заработает?

<Ответ>

**Вопрос 5**: Как сделать стайл трансфер для музыки? oO

<Ответ>

In [6]:
import lasagne
from theano import tensor as T
from lasagne.nonlinearities import *
from lasagne.layers import MaxPool2DLayer, InputLayer, Conv2DLayer as ConvLayer
from lasagne.layers import NonlinearityLayer
from lasagne.layers import ElemwiseSumLayer
from lasagne.layers import ExpressionLayer
from lasagne.layers import PadLayer
from lasagne.layers import GlobalPoolLayer
from lasagne.layers import DenseLayer

input_X = T.tensor4("X", dtype='float32')
target_y = T.vector("target Y integer",dtype='int32')  ##carefull! use int32 for gpu

### Соберите нейронку: 
- Many times x (Conv+Pool)
- Many small convolutions like 3x3
- Batch Norm 
- Residual Connection
- Data Augmentation 
- Learning rate Schedule 
- ...

### Для вдохновения 
- http://torch.ch/blog/2015/07/30/cifar.html
- http://www.robots.ox.ac.uk/~vgg/research/very_deep/
- https://www.cs.unc.edu/~wliu/papers/GoogLeNet.pdf
- https://github.com/szagoruyko/wide-residual-networks 

### Самое интересное
- Для сдачи задания нужно набрать на точность тесте > **92.5**% (это займет много времени, торопитесь :) )
- Для получения бонусных баллов > **95.0**%
- Будет очень хорошо если вы придумаете свою архитектуру или сможете обучить что-то из вышеперечисленного :)
- А для обучения всего этого добра вам будет куда удобнее использовать GPU на Amazon 
    - Инструкция https://github.com/persiyanov/ml-mipt/tree/master/amazon-howto 
    - Вам помогут tmux, CuDNN, ssh tunnel, nvidia-smi, ... 
    - Wish you get fun :)

In [7]:
depth = 3
depth_out = 2

In [8]:
def build_cnn(input_var=None, n=5):
    def residual_block(l, increase_dim=False, projection=False):
        input_num_filters = l.output_shape[1]
        if increase_dim:
            first_stride = (2,2)
            out_num_filters = input_num_filters*2
        else:
            first_stride = (1,1)
            out_num_filters = input_num_filters
        stack_1 = batch_norm(ConvLayer(l, num_filters=out_num_filters, filter_size=(3,3), stride=first_stride, nonlinearity=rectify, pad='same', W=lasagne.init.HeNormal(gain='relu'), flip_filters=False))
        stack_2 = batch_norm(ConvLayer(stack_1, num_filters=out_num_filters, filter_size=(3,3), stride=(1,1), nonlinearity=None, pad='same', W=lasagne.init.HeNormal(gain='relu'), flip_filters=False))
        if increase_dim:
            if projection:
                projection = batch_norm(ConvLayer(l, num_filters=out_num_filters, filter_size=(1,1), stride=(2,2), nonlinearity=None, pad='same', b=None, flip_filters=False))
                block = NonlinearityLayer(ElemwiseSumLayer([stack_2, projection]),nonlinearity=rectify)
            else:
                identity = ExpressionLayer(l, lambda X: X[:, :, ::2, ::2], lambda s: (s[0], s[1], s[2]//2, s[3]//2))
                padding = PadLayer(identity, [out_num_filters//4,0,0], batch_ndim=1)
                block = NonlinearityLayer(ElemwiseSumLayer([stack_2, padding]),nonlinearity=rectify)
        else:
            block = NonlinearityLayer(ElemwiseSumLayer([stack_2, l]),nonlinearity=rectify)
        return block
    l_in = InputLayer(shape=(None, 3, 32, 32), input_var=input_var)
    l = batch_norm(ConvLayer(l_in, num_filters=32, filter_size=(3,3), stride=(1,1), nonlinearity=rectify, pad='same', W=lasagne.init.HeNormal(gain='relu'), flip_filters=False))
    for _ in range(n):
        l = residual_block(l)
    l = residual_block(l, increase_dim=True)
    for _ in range(1,n):
        l = residual_block(l)
    l = residual_block(l, increase_dim=True)
    for _ in range(1,n):
        l = residual_block(l)
    l = GlobalPoolLayer(l)
    l = lasagne.layers.DenseLayer(l, num_units=64, W=lasagne.init.HeNormal(), nonlinearity=lasagne.nonlinearities.rectify)
    network = DenseLayer(
            l, num_units=10,
            W=lasagne.init.HeNormal(),
            nonlinearity=softmax)
    return network

In [9]:
# net = lasagne.layers.InputLayer(shape=(None, 3, 32, 32), input_var=input_X)

# for i in range(depth):
#     net = lasagne.layers.ConvLayer(net, 128 // (i+1), 3, pad='full', nonlinearity=lasagne.nonlinearities.rectify)
#     net = batch_norm(net)
#     net = MaxPool2DLayer(net, pool_size=(2, 2),stride=2,name="pool")
# for i in range(depth_out):
#     net = lasagne.layers.DenseLayer(net, num_units=512 // (i + 1), nonlinearity=lasagne.nonlinearities.rectify)
# net = lasagne.layers.DenseLayer(net,num_units = 10, nonlinearity=softmax)
net = build_cnn(input_X, 5)

In [10]:
y_predicted = lasagne.layers.get_output(net)
all_weights = lasagne.layers.get_all_params(net, trainable=True)
print all_weights

[W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, b, W, b]


In [11]:
loss = lasagne.objectives.categorical_crossentropy(y_predicted, target_y).mean()#<функция потерь>
all_layers = lasagne.layers.get_all_layers(net)
l2_penalty = lasagne.regularization.regularize_layer_params(all_layers, lasagne.regularization.l2) * 0.0001
loss = loss + l2_penalty
accuracy = lasagne.objectives.categorical_accuracy(y_predicted, target_y).mean()#<вычисление точност>

In [12]:
import sys
sys.setrecursionlimit(10000)

In [13]:
sh_lr = theano.shared(lasagne.utils.floatX(0.1))
updates = lasagne.updates.momentum(
                loss, all_weights, learning_rate=sh_lr, momentum=0.9)

In [14]:
train_fun = theano.function([input_X,target_y],[loss, accuracy], updates=updates)
accuracy_fun = theano.function([input_X,target_y],accuracy)

### Вот и всё, пошли её учить

In [ ]:
def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]

# Процесс обучения

In [46]:
import time

num_epochs = 60 #количество проходов по данным

batch_size = 256 #размер мини-батча

for epoch in range(num_epochs):
    # In each epoch, we do a full pass over the training data:
    train_err = 0
    train_acc = 0
    train_batches = 0
    start_time = time.time()
    for batch in iterate_minibatches(X_train.astype(np.float32),
                                     y_train.astype(np.float32),batch_size, shuffle=True):
        inputs, targets = batch
        inputs = inputs.astype(np.float32)
        targets = targets.astype(np.int32)
        train_err_batch, train_acc_batch= train_fun(inputs, targets)
        train_err += train_err_batch
        train_acc += train_acc_batch
        train_batches += 1

    # And a full pass over the validation data:
    val_acc = 0
    val_batches = 0
    print("Epoch {} of {} took {:.3f}s".format(epoch + 1, num_epochs, time.time() - start_time))
    if (epoch + 1) % 1 == 0:
        for batch in iterate_minibatches(X_test, y_test, batch_size):
            inputs, targets = batch
            targets = targets.astype(np.int32)
            val_acc += accuracy_fun(inputs, targets)
            val_batches += 1

        # Then we print the results for this epoch:
        print("Epoch {} of {} took {:.3f}s".format(epoch + 1, num_epochs, time.time() - start_time))
        print("  training loss (in-iteration):\t\t{:.6f}".format(train_err / train_batches))
        print("  train accuracy:\t\t{:.2f} %".format(train_acc / train_batches * 100))
        print("  validation accuracy:\t\t{:.2f} %".format(val_acc / val_batches * 100))

Epoch 1 of 60 took 193.731s
Epoch 1 of 60 took 198.118s
  training loss (in-iteration):		0.340322
  train accuracy:		97.59 %
  validation accuracy:		92.32 %


KeyboardInterrupt: 

In [45]:
test_acc = 0
test_batches = 0
for batch in iterate_minibatches(X_test, y_test, 500):
    inputs, targets = batch
    targets = targets.astype(np.int32)
    acc = accuracy_fun(inputs, targets)
    test_acc += acc
    test_batches += 1
print("Final results:")
print("  test accuracy:\t\t{:.2f} %".format(
    test_acc / test_batches * 100))

if test_acc / test_batches * 100 > 80:
    print "Achievement unlocked: колдун 80 уровня"
else:
    print "Нужно больше магии!"

Final results:
  test accuracy:		92.76 %
Achievement unlocked: колдун 80 уровня


## Заполните форму

https://goo.gl/forms/EeadABISlVmdJqgr2 